<a href="https://colab.research.google.com/github/edu-sabino/Python-Practices/blob/main/Webscraping_Stackoverflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#!pip install requests-html requests pandas

In [ ]:
import requests
from requests_html import HTML
import time
import pandas as pd

In [ ]:
def parse_tagged_page(html):
  question_summaries = html.find('.s-post-summary')
  key_names = ['id', 'question', 'date', 'tags', 'answers']
  data_needed = ['.data-post-id', '.s-link', '.s-user-card--time', '.s-post-summary--meta-tags', '.s-post-summary--stats-item']
  dados = []

  for q_element in question_summaries:
    question_data = {}
    for i, head in enumerate(data_needed):
      keyname = key_names[i]
      if head == '.data-post-id':
        sub_el = q_element.attrs['data-post-id'] # ex: 70574795
        question_data[keyname] = sub_el
     
      elif head == '.s-post-summary--stats-item':
        sub_el = q_element.find(head)
        question_data[keyname] = sub_el[1].text
     
      elif head == '.s-user-card--time':
        sub_el = q_element.find(head, first = True).text[6:12]
        question_data[keyname] = sub_el
      else:
        sub_el = q_element.find(head, first = True).text
        question_data[keyname] = sub_el
     #print(sub_el.text)
    dados.append(question_data)
  return dados

In [ ]:
def extract_data_from_url(url):
  r = requests.get(url) # usando o método GET para armazenar a página numa variável
  if r.status_code not in range(200, 299):
    return []
  html_str = r.text # exibindo o texto do HTML
  html = HTML(html=html_str) # Instanciando um objeto 'html' da classe HTML
  dados = parse_tagged_page(html)
  return dados

In [ ]:
def scrape_tag(tag='python', filter='newest', page=3360, max_pages=10):
  base_url = 'https://stackoverflow.com/questions/tagged/'
  dados = []
  for p in range(max_pages):
    page = 3360 + p # a medida que o tempo passa, este número vai aumentando
    url = f"{base_url}{tag}?tab={filter}&page={page}&pagesize=50"
    dados += extract_data_from_url(url)
    time.sleep(1.5)
  return dados

In [ ]:
dados_python_22 = scrape_tag()

In [ ]:
df = pd.DataFrame(dados_python_22)
df